# Drill - SQLite - Joins & CTEs

In [1]:
import sqlite3

In [2]:
file = 'bikeshare.db'
conn = sqlite3.connect(file)
c = conn.cursor()

# QUERY 1:  What are the three longest trips on rainy days?

In [3]:
q1 = c.execute('''
                WITH
                    rainy_days
                AS (
                    SELECT
                        Date
                    FROM
                        weather
                    WHERE
                        Events = "Rain"
                    GROUP BY 1
                )
                SELECT
                    trip_id,
                    duration,
                    DATE(start_date) trip_date
                FROM
                    trips
                JOIN
                    rainy_days
                ON
                    rainy_days.date = trip_date
                ORDER BY duration DESC
                LIMIT 3
                ''').fetchall()

In [4]:
q1

[('1173890', 85900, '2016-04-22'),
 ('1009870', 84349, '2015-11-15'),
 ('1210487', 83915, '2016-05-21')]

# QUERY 2:  Which station is full most often?

In [5]:
q2 = c.execute('''
               SELECT
                   status.station_id,
                   stations.name,
                   COUNT(CASE WHEN status.docks_available=0 then 1 END) empty_count
                FROM
                    status
                JOIN
                    stations
                ON
                    stations.station_id = status.station_id
                GROUP BY 1
                ORDER BY empty_count DESC
                ''').fetchall()

In [6]:
q2

[(70, 'San Francisco Caltrain (Townsend at 4th)', 23450),
 (73, 'Grant Avenue at Columbus Avenue', 12338),
 (69, 'San Francisco Caltrain 2 (330 Townsend)', 9852),
 (54, 'Embarcadero at Bryant', 9231),
 (50, 'Harry Bridges Plaza (Ferry Building)', 7590),
 (61, '2nd at Townsend', 6372),
 (60, 'Embarcadero at Sansome', 5894),
 (48, 'Embarcadero at Vallejo', 5868),
 (4, 'Santa Clara at Almaden', 5788),
 (65, 'Townsend at 7th', 5279),
 (63, 'Howard at 2nd', 4998),
 (84, 'Ryland Park', 4949),
 (64, '2nd at South Park', 4740),
 (74, 'Steuart at Market', 3992),
 (45, 'Commercial at Montgomery', 2916),
 (37, 'Cowper at University', 2402),
 (27, 'Mountain View City Hall', 2359),
 (57, '5th at Howard', 2282),
 (39, 'Powell Street BART', 1874),
 (36, 'California Ave Caltrain Station', 1745),
 (42, 'Davis at Jackson', 1608),
 (55, 'Temporary Transbay Terminal (Howard at Beale)', 1528),
 (3, 'San Jose Civic Center', 1444),
 (46, 'Washington at Kearney', 1342),
 (76, 'Market at 4th', 1251),
 (35, 'Un

# QUERY 3:  Return a list of stations with a count of number of trips starting at that station but ordered by dock count.

In [7]:
q3 = c.execute('''
                SELECT
                    trips.start_station,
                    stations.dockcount,
                    COUNT(*)
                FROM
                    trips
                JOIN
                    stations
                ON
                    stations.name = trips.start_station
                GROUP BY 1, 2
                ORDER BY 2 DESC
                ''').fetchall()

In [8]:
q3

[('Cyril Magnin St at Ellis St', 35, 69),
 ('5th St at Folsom St', 31, 173),
 ('2nd at Townsend', 27, 14099),
 ('Market at 10th', 27, 9937),
 ('Market at Sansome', 27, 10970),
 ('San Jose Diridon Caltrain Station', 27, 4035),
 ('Civic Center BART (7th at Market)', 23, 5043),
 ('Golden Gate at Polk', 23, 3309),
 ('Harry Bridges Plaza (Ferry Building)', 23, 16128),
 ('Mountain View Caltrain Station', 23, 2515),
 ('Palo Alto Caltrain Station', 23, 1170),
 ('San Antonio Caltrain Station', 23, 611),
 ('San Francisco Caltrain 2 (330 Townsend)', 23, 22358),
 ('Steuart at Market', 23, 13693),
 ('Temporary Transbay Terminal (Howard at Beale)', 23, 13111),
 ('2nd at Folsom', 19, 7464),
 ('Adobe on Almaden', 19, 789),
 ('Arena Green / SAP Center', 19, 547),
 ('Beale at Market', 19, 7373),
 ('Embarcadero at Folsom', 19, 7820),
 ('Grant Avenue at Columbus Avenue', 19, 6735),
 ('Howard at 2nd', 19, 7583),
 ('MLK Library', 19, 630),
 ('Market at 4th', 19, 7337),
 ('Mechanics Plaza (Market at Battery)

# QUERY 4:  (Challenge) What's the length of the longest trip for each day it rains anywhere?

In [9]:
q4 = c.execute('''
                WITH
                    rainy_days
                AS (
                    SELECT
                        Date
                    FROM
                        weather
                    WHERE
                        Events = "Rain"
                    GROUP BY 1
                ),
                    rainy_trips
                AS (
                    SELECT
                        trip_id,
                        duration,
                        DATE(start_date) trip_date
                    FROM
                        trips
                    JOIN
                        rainy_days
                    ON
                        rainy_days.date = trip_date
                    ORDER BY duration DESC
                )
                SELECT
                    trip_date,
                    MAX(duration)
                FROM
                    rainy_trips
                GROUP BY 1
               ''').fetchall()

In [10]:
q4

[('2015-09-02', 25917),
 ('2015-09-13', 21695),
 ('2015-09-16', 50241),
 ('2015-09-30', 25667),
 ('2015-10-01', 51081),
 ('2015-10-15', 27467),
 ('2015-10-16', 19073),
 ('2015-10-19', 41555),
 ('2015-10-27', 11107),
 ('2015-11-01', 43899),
 ('2015-11-02', 12246),
 ('2015-11-08', 7874),
 ('2015-11-09', 12838),
 ('2015-11-10', 61234),
 ('2015-11-12', 37799),
 ('2015-11-15', 84349),
 ('2015-11-24', 22478),
 ('2015-11-25', 3617),
 ('2015-11-30', 58759),
 ('2015-12-03', 16722),
 ('2015-12-06', 39450),
 ('2015-12-10', 8540),
 ('2015-12-11', 19186),
 ('2015-12-13', 57019),
 ('2015-12-18', 27880),
 ('2015-12-19', 33629),
 ('2015-12-20', 13739),
 ('2015-12-21', 18927),
 ('2015-12-22', 3999),
 ('2015-12-24', 8472),
 ('2015-12-27', 30361),
 ('2015-12-28', 7034),
 ('2015-12-30', 12416),
 ('2016-01-03', 61900),
 ('2016-01-04', 25318),
 ('2016-01-05', 5452),
 ('2016-01-06', 6401),
 ('2016-01-07', 28640),
 ('2016-01-08', 25855),
 ('2016-01-09', 11347),
 ('2016-01-10', 14749),
 ('2016-01-11', 26442),


In [11]:
conn.close()